# Imports

In [30]:
import sys
import pandas as pd
import requests
import qgrid
import random
from textblob import TextBlob
import spotipy
import spotify_cred
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from IPython.display import IFrame

# Song recommendation functions

In [84]:
def get_user_input():
    your_fav_song_raw=input("Please share a song you enjoy to receive a recommendation!")
    your_fav_song=your_fav_song_raw.lower()
    return your_fav_song


def check_duplicate_song_names(your_fav_song):
    num_songs=sp.search(q="track:"+your_fav_song, limit=3)
    
    #Check if there are more than one artist that sings that song
    if len(num_songs['tracks']['items'])>1:
        artist_list=[]
        for i in num_songs['tracks']['items']:
            for j in i['artists']:
                artist_list.append(j['name'])
                artist_list=list(set(artist_list))
                artist_string=", ".join(artist_list)
        if len(artist_list)>1:
            print("There is more than one artist with a song title like",your_fav_song.title()) 
            print(f"Similar song titles are offered by the following artists: {artist_string}")
            print("...")

            #Gather user input on correct artist.  Check it against the list of artists, error if not in list. 
            matched=False
            while matched==False:
                user_clarification=input("Which artist is your song by?")

            #Compare to ensure input is good
                song_check=sp.search(q="track:"+your_fav_song+" artist:"+user_clarification,limit=1)
                try:
                    x=song_check['tracks']['items'][0]['id']
                    matched=True
                except:
                    print("...")
                    print(f"{your_fav_song} by {user_clarification} was not found in list, please try again")
        else:
            user_clarification=artist_string
    else:
        user_clarification=[j['name'] for j in num_songs['tracks']['items'][0]['artists']]
    return user_clarification.lower()

def return_recommendation(fav_song_spotify_id,your_fav_song,top_100_ids,top_100):
    top_99_songs=top_100_ids
    top_99_songs.remove(fav_song_spotify_id)
    recommended_song_id=random.choice(top_99_songs)
    recommended_song=top_100.loc[top_100['id']==recommended_song_id,'song_name'].item()
    recommended_artist=top_100.loc[top_100['id']==recommended_song_id,'artist_name'].item()
    your_fav_song=your_fav_song.title()
    print(f"{your_fav_song} is currently in the top 100! Maybe you'll also like this top 100 song: {recommended_song}, by {recommended_artist}")
    return recommended_song_id
    
def spotipy_login():
    c_id = spotify_cred.c_id
    c_se = spotify_cred.c_se

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))
    return sp

def load_pickle(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 
        
def get_fav_song_audio_features(fav_song_audio_features):
    
    fsaf=pd.DataFrame(fav_song_audio_features)
    fsaf= fsaf[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id"]]

    artists=[]
    song_titles=[]
    popularity=[]
    explicit=[]

    for ids in fsaf['id']:
        temp_val=sp.track(ids)
        song_titles.append(temp_val['name'])
        popularity.append(temp_val['popularity'])
        explicit.append(temp_val['explicit'])
        temp_artist=[artist['name'] for artist in temp_val['artists']]
        artists.append([temp_artist])

    fsaf['explicit']=explicit
    fsaf['popularity']=popularity
    fsaf.drop(labels='id',axis=1,inplace=True)
    return fsaf

def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

def recommender(fav_song_spotify_id):
    #Get top songs
    top_100=pd.read_csv(r"Data\top100_ids.csv")

    #Lowercase all values for comparison
    top_100_songs=list(top_100['song_name'])
    top_100_ids=list(top_100['id'])


    #If song currently hot...

    #Give another hot song as output
    if fav_song_spotify_id in top_100_ids:
        recommendation=return_recommendation(fav_song_spotify_id,your_fav_song,top_100_ids,top_100)
        return play_song(recommendation)
    else:
        #If song not hot...

        # Get audio values for individual song
        fav_song_audio_features=sp.audio_features(fav_song_spotify_id)
        fsaf=get_fav_song_audio_features(fav_song_audio_features)

        #Run through predict of k-means to get cluster #
        scaler=load_pickle("Model/scaler25.pickle")
        kmeans=load_pickle("Model/kmeans25.pickle")
        fsaf_scaled=scaler.transform(fsaf)
        fsaf_scaled_df = pd.DataFrame(fsaf_scaled, columns = fsaf.columns)
        fav_song_cluster=kmeans.predict(fsaf_scaled_df)
        #Load database of songs with clusters
        song_database=pd.read_csv("Data\songs_with_clusters25.csv")

        #Get sample from database
        song_database=song_database[['id','song_title','artist','clusters']]
        recommendation=song_database[song_database['clusters']==fav_song_cluster[0]].sample()
        rec_artist=recommendation['artist'].iat[0]
        rec_song=recommendation['song_title'].iat[0]
        print("...")
        print(f"You have some unique taste! Maybe you'll also like: {rec_song} by {rec_artist}")
        print("...")

        return play_song(recommendation['id'].iat[0])

def get_fav_song_spotify_id(your_fav_song,artist_name):
    fav_song_spotify=sp.search(q="track:"+your_fav_song+" artist:"+artist_name,limit=5)
    fav_song_spotify_id=fav_song_spotify['tracks']['items'][0]['id']
    return fav_song_spotify_id

# Main function

In [88]:
#Login to spotipy
sp=spotipy_login()

#Get user input
your_fav_song=get_user_input()

#Check if user song is unique, if not ask for clarification
artist_name=check_duplicate_song_names(your_fav_song)

#Get id for fav_song from spotify
fav_song_spotify_id=get_fav_song_spotify_id(your_fav_song,artist_name)

recommender(fav_song_spotify_id)

Please share a song you enjoy to receive a recommendation! wagon wheel


There is more than one artist with a song title like Wagon Wheel
Similar song titles are offered by the following artists: daigoro789, Old Crow Medicine Show, Darius Rucker
...


Which artist is your song by? darius rucker


...
You have some unique taste! Maybe you'll also like: This Charming Man - 2011 Remaster by The Smiths
...


In [89]:
play_song(fav_song_spotify_id)